### Upsample raw data to balance with respect to TARGET_5Yrs variable.

Note this data has not bee scaled ad is prepared to be a precise comparison with SMOTE data for random forest experiments.

follow this tutorial:
https://wellsr.com/python/upsampling-and-downsampling-imbalanced-data-in-python/

In [33]:
import pandas as pd
import numpy as np

from joblib import dump
from joblib import load

from sklearn.utils import resample

from sklearn.model_selection import train_test_split

In [2]:
#Read full, raw, labelled training data

df = pd.read_csv('../data/raw/2022_train.csv')

In [3]:
# Copy full data set to df_cleaned 

df_cleaned = df.copy()

In [4]:
df_cleaned.drop('Id', axis=1, inplace=True)

In [5]:
list(df_cleaned.columns)

['GP',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3P Made',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'TARGET_5Yrs']

In [6]:
TARGET_5Yrs = df_cleaned.pop('TARGET_5Yrs')

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_data, X_test, y_data, y_test = train_test_split (df_cleaned, TARGET_5Yrs, test_size=0.2, random_state=8)

In [12]:
# rejoin X_data and y_data for upsampling

Xy_data = pd.concat([X_data, y_data], axis=1)

In [13]:
Xy_data.groupby('TARGET_5Yrs').size()

TARGET_5Yrs
0    1074
1    5326
dtype: int64

In [15]:
# split the raw data into two sets using the TARGET_5Yrs variable.

TARGET_0 = Xy_data[Xy_data["TARGET_5Yrs"] == 0]
TARGET_1 = Xy_data[Xy_data["TARGET_5Yrs"] == 1]
print(TARGET_0.shape)
print(TARGET_1.shape)

(1074, 20)
(5326, 20)


In [16]:
# upsample TARGET_0

TARGET_0_ups = resample(TARGET_0, replace=True, n_samples = len(TARGET_1), random_state = 8)

print(TARGET_0_ups.shape)

(5326, 20)


In [35]:
Xy_data = pd.concat([TARGET_1, TARGET_0_ups])

print(Xy_data.shape)
print(Xy_data.groupby('TARGET_5Yrs').size())

(10652, 20)
TARGET_5Yrs
0    5326
1    5326
dtype: int64


In [36]:
y_data_ups = Xy_data.pop('TARGET_5Yrs')
X_data_ups = Xy_data

print(X_data_ups.shape)
print(y_data_ups.shape)

(10652, 19)
(10652,)


In [37]:
np.save('../data/processed/X_data_ups', X_data_ups)
np.save('../data/processed/y_data_ups', y_data_ups)